# Summarize Results

This notebook reads the results from each of the finalized shapefiles and writes the results to a .csv file

In [1]:
import glob
import os
import os.path as op
import sys
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import Polygon
import zipfile

# set working dir
HOME = op.join(op.expanduser("~"))
os.chdir(os.path.join(HOME, "git/wgms-glacier-project"))

# Set glacier and ice catchment region numbers
region = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', 
          '12', '13', '14', '15', '16', '17', '18', '19']
#catch_region = ['3', '4', '5', '6', '7', '8', '9', '10', '17', '19']

# To include the clipped ice caps set this to 1 to exclude them set to 0 
# Note as of Aug 2021, decided to not use the clipped catchments
include_clipped = 0

## Glaciers

In [2]:
# Unzip the glacier shapefiles
for x in region:
    # Read Region 19 separately because they have a different naming convention for mainland and islands
    if x == "19":
        # Unzip mainland glacier file
        mainland_glacier_fp = "data/final-dataset/region-" + x + "-mainland-largest-glaciers.zip"
        with zipfile.ZipFile(mainland_glacier_fp,"r") as zip_ref:
            zip_ref.extractall("data/final-dataset/unzipped")
            
        # Unxip island glacier file
        island_glacier_fp = "data/final-dataset/region-" + x + "-islands-largest-glaciers.zip"
        with zipfile.ZipFile(island_glacier_fp,"r") as zip_ref:
            zip_ref.extractall("data/final-dataset/unzipped")
    else:
        # Unzip all other region glacier files
        glacier_zipfile_fn = "data/final-dataset/region-" + x + "-largest-glaciers.zip"
        with zipfile.ZipFile(glacier_zipfile_fn,"r") as zip_ref:
            zip_ref.extractall("data/final-dataset/unzipped")
        


In [3]:
# Open finalized glacier shapefiles and concatenate them to a single data frame
for x in region:
    # Read the first region
    if x == '1':
        glacier_shapefile_fn = "data/final-dataset/unzipped/region-" + x + "-largest-glaciers.shp"
        glacier_regions = gpd.read_file(glacier_shapefile_fn)

    # Read Regioin 19 files with the different naming convention
    elif x == "19":
        # Read mainland glacier file
        mainland_glacier_shapefile = "data/final-dataset/unzipped/region-" + x + \
        "-mainland-largest-glaciers.shp"
        glacier_regions_part = gpd.read_file(mainland_glacier_shapefile)
        glacier_regions_part['reg_name'] = 'Antarctic Mainland'
        glacier_regions = glacier_regions.append(glacier_regions_part)
        
        # Read island glacier file
        island_glacier_shapefile = "data/final-dataset/unzipped/region-" + x + \
        "-islands-largest-glaciers.shp"
        glacier_regions_part = gpd.read_file(island_glacier_shapefile)
        glacier_regions_part['reg_name'] = 'Subantarctic Islands'
        glacier_regions = glacier_regions.append(glacier_regions_part)  
   
    # Read all the other regions
    else:
        glacier_shapefile_fn = "data/final-dataset/unzipped/region-" + x + "-largest-glaciers.shp"
        glacier_regions_part = gpd.read_file(glacier_shapefile_fn)
        glacier_regions = glacier_regions.append(glacier_regions_part)

In [4]:
# Write glacier dataframe to csv
csv_glacier_fp = "data/final-dataset/compiled-glacier-sizes.csv"
if os.path.exists(csv_glacier_fp) == False:
    glacier_regions.to_csv(csv_glacier_fp, encoding='utf-8-sig', 
                           index=False, columns=['region_no', 'reg_name', 'glac_name', 
                                                 'glims_id', 'primeclass', 'area_km2', 'date'])
    print("Creating csv file: " + csv_glacier_fp)
else:
    print(csv_glacier_fp + " already extists")

Creating csv file: data/final-dataset/compiled-glacier-sizes.csv


## Ice Catchments

In [5]:
# Unzip the ice catchment shapefiles
for x in region:
    # Set up special case for Region 19 - Antarctica with the different naming convention for mainland and islands
    # and for the inclusion of clipped ice caps
    if x == "19":
        # Unzip mainland ice cap file
        ic_zipfile_fn = "data/final-dataset/region-" + x + "-mainland-largest-complexes.zip"
        
        with zipfile.ZipFile(ic_zipfile_fn,"r") as zip_ref:
            zip_ref.extractall("data/final-dataset/unzipped")

        # Unzip island ice cap file
        ic_zipfile_fn = "data/final-dataset/region-" + x + "-islands-largest-complexes.zip"
        
        with zipfile.ZipFile(ic_zipfile_fn,"r") as zip_ref:
            zip_ref.extractall("data/final-dataset/unzipped")
            
        # Unzip clipped file
        if include_clipped == 1:
            ic_zipfile_fn = "data/final-dataset/region-" + x + "-islands-largest-complexes-clipped.zip"
        
            with zipfile.ZipFile(ic_zipfile_fn,"r") as zip_ref:
                zip_ref.extractall("data/final-dataset/unzipped")
                
    # Special statemnet for regions 5, 7, and 9 which have clipped versions of the ice caps
    elif x == "5" or x == "7" or x == "9":
        # Unzip regular ice cap files for these regions
        ic_zipfile_fn = "data/final-dataset/region-" + x + "-largest-complexes.zip"
        
        with zipfile.ZipFile(ic_zipfile_fn,"r") as zip_ref:
            zip_ref.extractall("data/final-dataset/unzipped")
            
        # Unzip the clipped ice cap files for these regions
        if include_clipped == 1:
            ic_zipfile_fn = "data/final-dataset/region-" + x + "-largest-complexes-clipped.zip"
        
            with zipfile.ZipFile(ic_zipfile_fn,"r") as zip_ref:
                zip_ref.extractall("data/final-dataset/unzipped")
                
    # Unzip all other regions ice cap files
    else: 
        ic_zipfile_fn = "data/final-dataset/region-" + x + "-largest-complexes.zip"
        
        with zipfile.ZipFile(ic_zipfile_fn,"r") as zip_ref:
            zip_ref.extractall("data/final-dataset/unzipped")

In [6]:
# Open finalized ice catchment shapefiles and concatenate them to a single data frame
for x in region:
    ic_shapefile_fn = "data/final-dataset/unzipped/region-" + x + "-largest-complexes.shp"
    
    # Read first shapefile to set up dataframe
    if x == "1":
        ic_regions = gpd.read_file(ic_shapefile_fn)
        ic_regions['clipped'] = 0
        
    # Read region 19 shapefile
    elif x == "19":
        # Read mainland ice cap file
        ic_shapefile_fn = "data/final-dataset/unzipped/region-" + x + "-mainland-largest-complexes.shp"
        ic_regions_part = gpd.read_file(ic_shapefile_fn)
        ic_regions_part['clipped'] = 0
        ic_regions_part['reg_name'] = 'Antarctic Mainland'
        ic_regions = ic_regions.append(ic_regions_part)
        
        # Read island ice cap file
        ic_shapefile_fn = "data/final-dataset/unzipped/region-" + x + "-islands-largest-complexes.shp"
        ic_regions_part = gpd.read_file(ic_shapefile_fn)
        ic_regions_part['clipped'] = 0
        ic_regions_part['reg_name'] = 'Subantarctic Islands'
        ic_regions = ic_regions.append(ic_regions_part)
        
        # Read island clipped ice cap file
        if include_clipped == 1:
            ic_shapefile_fn = "data/final-dataset/unzipped/region-" + x + "-islands-largest-complexes-clipped.shp"
            ic_regions_part = gpd.read_file(ic_shapefile_fn)
            ic_regions_part['clipped'] = 1
            # drop rgi_ids column
            #ic_regions_part = ic_regions_rgi.drop(['rgi_ids'], axis=1)
            ic_regions = ic_regions.append(ic_regions_part)
    
    # Read regions 5, 7, and 9 and the clipped versions as well if desired
    elif x == "5" or x == "7" or x == "9":
        # Read the regular ice cap files for these regions
        ic_shapefile_fn = "data/final-dataset/unzipped/region-" + x + "-largest-complexes.shp"
        ic_regions_part = gpd.read_file(ic_shapefile_fn)
        ic_regions_part['clipped'] = 0
        ic_regions = ic_regions.append(ic_regions_part)
        
        # Read the clipped versions of these ice cap files for these regions
        if include_clipped == 1:
            ic_shapefile_fn = "data/final-dataset/unzipped/region-" + x + "-largest-complexes-clipped.shp"        
            ic_regions_part = gpd.read_file(ic_shapefile_fn)
            ic_regions_part['clipped'] = 1
            ic_regions = ic_regions.append(ic_regions_part)

    # Read all the other region shapefiles
    else:
        ic_regions_part = gpd.read_file(ic_shapefile_fn)
        ic_regions_part['clipped'] = 0
        ic_regions = ic_regions.append(ic_regions_part)

In [7]:
ic_regions[50:57]

,region_no,reg_name,ic_name,primeclass,area_km2,min_date,max_date,geometry,clipped
2,17,Southern Andes,Cordillera Darwin Icefield,2,1893.579143,2000-06-15,2007-09-06,"(POLYGON ((-70.10073199999999 -54.740177, -70....",0
0,18,New Zealand,Tasman Glacier Complex,2,248.785404,1978-01-01,2009-02-17,(POLYGON ((170.1329175306282 -43.5884397947709...,0
1,18,New Zealand,Adams & Lambert Glacier Complex,2,42.952788,1978-01-01,1978-01-01,"POLYGON ((170.753766 -43.31865, 170.753756 -43...",0
2,18,New Zealand,Lyell & Ramsay Glacier Complex,2,31.190311,1978-01-01,1978-01-01,"POLYGON ((170.802738 -43.316774, 170.802939 -4...",0
0,19,Antarctic Mainland,Antarctic Penisula Glacier Complex,2,80851.900280,2002-12-31,2002-12-31,"POLYGON ((-65.016774 -67.351969, -65.027069 -6...",0
0,19,Subantarctic Islands,Alexander Island Glacier Complex,2,47486.230388,1979-02-99,2001-12-20,POLYGON ((-68.65115005999996 -72.2207072749999...,0
1,19,Subantarctic Islands,Thurston Island Ice Cap,3,11133.020670,1972-12-04,1972-12-04,POLYGON ((-96.38988303899998 -72.2732756079999...,0


In [8]:
# Write ice complex dataframe to csv
csv_catchment_fp = "data/final-dataset/compiled-complex-sizes.csv"
if os.path.exists(csv_catchment_fp) == False:
    ic_regions.to_csv(csv_catchment_fp, encoding='utf-8-sig', index=False, columns=['region_no', 'reg_name', 
                                                                                    'ic_name', 'primeclass', 
                                                                                    'area_km2', 'min_date',
                                                                                    'max_date', 'clipped'])
                                                                                    
    print("Creating csv file: " + csv_catchment_fp)
else:
    print(csv_catchment_fp + " already extists")

data/final-dataset/compiled-complex-sizes.csv already extists


## Clean up

In [9]:
# Clean up unzipped files to save disk space
filelist = glob.glob("data/final-dataset/unzipped/*")
for f in filelist:
    os.remove(f)